In [2]:
import sys

sys.path.append('..')


In [14]:
import os
import json
import argparse
import pandas as pd
from natsort import natsorted
from tqdm import tqdm
from qiskit import QuantumCircuit
import bqskit
from bqskit.compiler import Compiler
from bqskit.passes import QuickPartitioner
from phoenix import Circuit, gates
from phoenix.utils.ops import is_tensor_prod

from scipy.stats import gmean



### SU(4) ISA

In [109]:
def su4_circ_stats(qasm_fname):
    """Statistic of #2Q and Depth-2Q of SU(4)-based circuit."""
    circ = bqskit.Circuit.from_file(qasm_fname)
    blocks = list(bqskit_compiler.compile(circ, workflow))
    circ_su4 = Circuit(
        [gates.X.on(blk.location[0], blk.location[1]) for blk in blocks if blk.get_unitary().numpy.shape == (4, 4) and not is_tensor_prod(blk.get_unitary().numpy)])
    return circ_su4.num_gates, circ_su4.depth


In [ ]:

bqskit_compiler = Compiler()
workflow = QuickPartitioner(2)


selected_benchmarks = [
    'CH2_cmplt_BK',
    'CH2_cmplt_JW',
    'CH2_frz_BK',
    'CH2_frz_JW',
    'H2O_cmplt_BK',
    'H2O_cmplt_JW',
    'H2O_frz_BK',
    'H2O_frz_JW',
    'LiH_cmplt_BK',
    'LiH_cmplt_JW',
    'LiH_frz_BK',
    'LiH_frz_JW',
    'NH_cmplt_BK',
    'NH_cmplt_JW',
    'NH_frz_BK',
    'NH_frz_JW'
]

OUTPUT_DPATH = './output_uccsd/'

result = pd.DataFrame(columns=['program', 'num_su4(tket)', 'depth_su4(tket)',
                               'num_su4(paulihedral)', 'depth_su4(paulihedral)',
                               'num_su4(tetris)', 'depth_su4(tetris)',
                               'num_su4(phoenix)', 'depth_su4(phoenix)'])
result['program'] = selected_benchmarks

for compiler in ['tket', 'paulihedral', 'tetris', 'phoenix']:
    num_su4_list = []
    depth_su4_list = []
    output_dpath = os.path.join(OUTPUT_DPATH, compiler, 'all2all_opt')
    print(f'Processing {output_dpath}...')
    for program_name in tqdm(selected_benchmarks):
        fname = os.path.join(output_dpath, program_name + '_sto3g' + '.qasm')
        num_su4, depth_su4 = su4_circ_stats(fname)
        num_su4_list.append(num_su4)
        depth_su4_list.append(depth_su4)

    result[f'num_su4({compiler})'] = num_su4_list
    result[f'depth_su4({compiler})'] = depth_su4_list

bqskit_compiler.close()

Processing ./output_uccsd/tket/all2all_opt...


100%|██████████| 16/16 [00:28<00:00,  1.77s/it]


Processing ./output_uccsd/paulihedral/all2all_opt...


100%|██████████| 16/16 [00:21<00:00,  1.32s/it]


Processing ./output_uccsd/tetris/all2all_opt...


100%|██████████| 16/16 [00:32<00:00,  2.03s/it]


Processing ./output_uccsd/phoenix/all2all_opt...


100%|██████████| 16/16 [00:17<00:00,  1.12s/it]


In [ ]:
# result_1 = pd.DataFrame(columns=['program', 'num_su4(tket)', 'depth_su4(tket)',
#                                'num_su4(paulihedral)', 'depth_su4(paulihedral)',
#                                'num_su4(tetris)', 'depth_su4(tetris)',
#                                'num_su4(phoenix)', 'depth_su4(phoenix)'])
# result_1['program'] = selected_benchmarks

# bqskit_compiler = Compiler()

# for compiler in ['phoenix']:
#     num_su4_list = []
#     depth_su4_list = []
#     output_dpath = os.path.join(OUTPUT_DPATH, compiler, 'all2all')
#     print(f'Processing {output_dpath}...')
#     for program_name in tqdm(selected_benchmarks):
#         fname = os.path.join(output_dpath, program_name + '_sto3g' + '.qasm')
#         num_su4, depth_su4 = su4_circ_stats(fname)
#         num_su4_list.append(num_su4)
#         depth_su4_list.append(depth_su4)

#     result_1[f'num_su4({compiler})'] = num_su4_list
#     result_1[f'depth_su4({compiler})'] = depth_su4_list

# bqskit_compiler.close()

Processing ./output_uccsd/phoenix/all2all...


100%|██████████| 16/16 [00:36<00:00,  2.29s/it]


In [102]:
result

,program,num_su4(tket),depth_su4(tket),num_su4(paulihedral),depth_su4(paulihedral),num_su4(tetris),depth_su4(tetris),num_su4(phoenix),depth_su4(phoenix)
0,CH2_cmplt_BK,6454,5787,4617,4601,6013,5847,3537,2969
1,CH2_cmplt_JW,6134,5282,4126,4062,6235,5936,3486,2874
2,CH2_frz_BK,3636,3276,3038,2998,3298,3285,1904,1634
3,CH2_frz_JW,2942,2593,2170,2162,3344,3067,1876,1586
4,H2O_cmplt_BK,4678,4131,3051,3037,4092,4083,2371,2004
5,H2O_cmplt_JW,4336,3783,2800,2798,4228,3922,2338,1967
6,H2O_frz_BK,2937,2605,2008,1991,2314,2300,1257,1051
7,H2O_frz_JW,2576,2221,1652,1647,2583,2356,1473,1250
8,LiH_cmplt_BK,2867,2638,2319,2293,2315,2250,1308,1127
9,LiH_cmplt_JW,2104,1842,1682,1673,2537,2308,1444,1228


In [111]:

bqskit_compiler = Compiler()
workflow = QuickPartitioner(2)


selected_benchmarks = [
    'CH2_cmplt_BK',
    'CH2_cmplt_JW',
    'CH2_frz_BK',
    'CH2_frz_JW',
    'H2O_cmplt_BK',
    'H2O_cmplt_JW',
    'H2O_frz_BK',
    'H2O_frz_JW',
    'LiH_cmplt_BK',
    'LiH_cmplt_JW',
    'LiH_frz_BK',
    'LiH_frz_JW',
    'NH_cmplt_BK',
    'NH_cmplt_JW',
    'NH_frz_BK',
    'NH_frz_JW'
]

OUTPUT_DPATH = './output_uccsd/'

result_manhattan = pd.DataFrame(columns=['program', 'num_su4(tket)', 'depth_su4(tket)',
                               'num_su4(paulihedral)', 'depth_su4(paulihedral)',
                               'num_su4(tetris)', 'depth_su4(tetris)',
                               'num_su4(phoenix)', 'depth_su4(phoenix)'])
result_manhattan['program'] = selected_benchmarks

for compiler in ['tket', 'paulihedral', 'tetris', 'phoenix']:
    num_su4_list = []
    depth_su4_list = []
    output_dpath = os.path.join(OUTPUT_DPATH, compiler, 'manhattan')
    print(f'Processing {output_dpath}...')
    for program_name in tqdm(selected_benchmarks):
        fname = os.path.join(output_dpath, program_name + '_sto3g' + '.qasm')
        num_su4, depth_su4 = su4_circ_stats(fname)
        num_su4_list.append(num_su4)
        depth_su4_list.append(depth_su4)

    result_manhattan[f'num_su4({compiler})'] = num_su4_list
    result_manhattan[f'depth_su4({compiler})'] = depth_su4_list

bqskit_compiler.close()

Processing ./output_uccsd/tket/manhattan...


100%|██████████| 16/16 [01:22<00:00,  5.14s/it]


Processing ./output_uccsd/paulihedral/manhattan...


100%|██████████| 16/16 [01:20<00:00,  5.02s/it]


Processing ./output_uccsd/tetris/manhattan...


100%|██████████| 16/16 [01:02<00:00,  3.89s/it]


Processing ./output_uccsd/phoenix/manhattan...


100%|██████████| 16/16 [00:40<00:00,  2.54s/it]


In [113]:
result_manhattan

,program,num_su4(tket),depth_su4(tket),num_su4(paulihedral),depth_su4(paulihedral),num_su4(tetris),depth_su4(tetris),num_su4(phoenix),depth_su4(phoenix)
0,CH2_cmplt_BK,14123,8704,13286,10414,8903,7176,6285,4502
1,CH2_cmplt_JW,13500,8551,15027,13719,9270,7633,5690,4180
2,CH2_frz_BK,7529,4872,7629,6199,4758,3912,3196,2359
3,CH2_frz_JW,6048,3908,7826,6979,4589,3734,2880,2190
4,H2O_cmplt_BK,9722,6378,9455,8132,5601,4543,4138,3005
5,H2O_cmplt_JW,9261,6097,11589,10482,6417,5215,3813,2761
6,H2O_frz_BK,5908,3867,5521,4639,3050,2589,1986,1541
7,H2O_frz_JW,5051,3468,5818,5428,4078,3425,2317,1812
8,LiH_cmplt_BK,6000,4082,5808,5028,2982,2399,2103,1611
9,LiH_cmplt_JW,4280,2893,6051,5071,3640,2985,2289,1737


In [116]:
# All2all

print('number of SU(4) gates')

print('reduction rates v.s. tket, {:.2f}%'.format(gmean(result['num_su4(phoenix)'] / result['num_su4(tket)']) * 100))
print('reduction rates v.s. paulihedral: {:.2f}%'.format(gmean(result['num_su4(phoenix)'] / result['num_su4(paulihedral)']) * 100))
print('reduction rates v.s. tetris: {:.2f}%'.format(gmean(result['num_su4(phoenix)'] / result['num_su4(tetris)']) * 100))

print('depth of SU(4) gates') 
print('reduction rates v.s. tket: {:.2f}%'.format(gmean(result['depth_su4(phoenix)'] / result['depth_su4(tket)']) * 100))
print('reduction rates v.s. paulihedral: {:.2f}%'.format(gmean(result['depth_su4(phoenix)'] / result['depth_su4(paulihedral)']) * 100))
print('reduction rates v.s. tetris: {:.2f}%'.format(gmean(result['depth_su4(phoenix)'] / result['depth_su4(tetris)']) * 100))

number of SU(4) gates
reduction rates v.s. tket, 56.12%
reduction rates v.s. paulihedral: 75.69%
reduction rates v.s. tetris: 56.63%
depth of SU(4) gates
reduction rates v.s. tket: 54.21%
reduction rates v.s. paulihedral: 65.18%
reduction rates v.s. tetris: 50.54%


In [117]:
# Manhattan

print('number of SU(4) gates')

print('reduction rates v.s. tket, {:.2f}%'.format(gmean(result_manhattan['num_su4(phoenix)'] / result_manhattan['num_su4(tket)']) * 100))
print('reduction rates v.s. paulihedral: {:.2f}%'.format(gmean(result_manhattan['num_su4(phoenix)'] / result_manhattan['num_su4(paulihedral)']) * 100))
print('reduction rates v.s. tetris: {:.2f}%'.format(gmean(result_manhattan['num_su4(phoenix)'] / result_manhattan['num_su4(tetris)']) * 100))

print('depth of SU(4) gates') 
print('reduction rates v.s. tket: {:.2f}%'.format(gmean(result_manhattan['depth_su4(phoenix)'] / result_manhattan['depth_su4(tket)']) * 100))
print('reduction rates v.s. paulihedral: {:.2f}%'.format(gmean(result_manhattan['depth_su4(phoenix)'] / result_manhattan['depth_su4(paulihedral)']) * 100))
print('reduction rates v.s. tetris: {:.2f}%'.format(gmean(result_manhattan['depth_su4(phoenix)'] / result_manhattan['depth_su4(tetris)']) * 100))

number of SU(4) gates
reduction rates v.s. tket, 44.44%
reduction rates v.s. paulihedral: 39.97%
reduction rates v.s. tetris: 62.44%
depth of SU(4) gates
reduction rates v.s. tket: 50.65%
reduction rates v.s. paulihedral: 35.02%
reduction rates v.s. tetris: 58.66%


In [118]:
result.to_csv('result_su4_all2all.csv', index=False)
result_manhattan.to_csv('result_su4_manhattan.csv', index=False)

### CNOT ISA

In [2]:
import pandas as pd
from scipy.stats import gmean

In [3]:
result_tket = pd.read_csv('./results/result_uccsd_tket.csv')
result_paulihedral = pd.read_csv('./results/result_uccsd_paulihedral.csv')
result_tetris = pd.read_csv('./results/result_uccsd_tetris.csv')
result_phoenix = pd.read_csv('./results/result_uccsd_phoenix.csv')


In [6]:
# All2all

print('number of CNOT gates')

print('reduction rates v.s. tket, {:.2f}%'.format(gmean(result_phoenix['num_2q_gates(all2all)'] / result_tket['num_2q_gates(all2all_opt)']) * 100))
print('reduction rate v.s. paulihedral, {:.2f}%'.format(gmean(result_phoenix['num_2q_gates(all2all)'] / result_paulihedral['num_2q_gates(all2all_opt)']) * 100))
print('reduction rate v.s. tetris, {:.2f}%'.format(gmean(result_phoenix['num_2q_gates(all2all)'] / result_tetris['num_2q_gates(all2all_opt)']) * 100))

print('depth of CNOT gates')
print('reduction rate v.s. tket, {:.2f}%'.format(gmean(result_phoenix['depth_2q(all2all)'] / result_tket['depth_2q(all2all_opt)']) * 100))
print('reduction rate v.s. paulihedral, {:.2f}%'.format(gmean(result_phoenix['depth_2q(all2all)'] / result_paulihedral['depth_2q(all2all_opt)']) * 100))
print('reduction rate v.s. tetris, {:.2f}%'.format(gmean(result_phoenix['depth_2q(all2all)'] / result_tetris['depth_2q(all2all_opt)']) * 100))


number of CNOT gates
reduction rates v.s. tket, 63.93%
reduction rate v.s. paulihedral, 82.21%
reduction rate v.s. tetris, 57.58%
depth of CNOT gates
reduction rate v.s. tket, 63.96%
reduction rate v.s. paulihedral, 73.29%
reduction rate v.s. tetris, 53.01%


In [122]:
# Manhattan

print('number of CNOT gates')

print('reduction rates v.s. tket, {:.2f}%'.format(gmean(result_phoenix['num_2q_gates(manhattan)'] / result_tket['num_2q_gates(manhattan)']) * 100))
print('reduction rate v.s. paulihedral, {:.2f}%'.format(gmean(result_phoenix['num_2q_gates(manhattan)'] / result_paulihedral['num_2q_gates(manhattan)']) * 100))
print('reduction rate v.s. tetris, {:.2f}%'.format(gmean(result_phoenix['num_2q_gates(manhattan)'] / result_tetris['num_2q_gates(manhattan)']) * 100))

print('depth of CNOT gates')
print('reduction rate v.s. tket, {:.2f}%'.format(gmean(result_phoenix['depth_2q(manhattan)'] / result_tket['depth_2q(manhattan)']) * 100))
print('reduction rate v.s. paulihedral, {:.2f}%'.format(gmean(result_phoenix['depth_2q(manhattan)'] / result_paulihedral['depth_2q(manhattan)']) * 100))
print('reduction rate v.s. tetris, {:.2f}%'.format(gmean(result_phoenix['depth_2q(manhattan)'] / result_tetris['depth_2q(manhattan)']) * 100))


number of CNOT gates
reduction rates v.s. tket, 40.79%
reduction rate v.s. paulihedral, 62.63%
reduction rate v.s. tetris, 76.27%
depth of CNOT gates
reduction rate v.s. tket, 48.51%
reduction rate v.s. paulihedral, 54.92%
reduction rate v.s. tetris, 71.47%
